# Orbital-Optimization for reducing the one-norm

Author : Dr. Saad Yalouz

A starting point to minimize the One-norm using Orbital-Optimization and a gradient free optimizer

## Theory 
The idea here is to employ a unitary transformation to modify the MOs of the system in order to minimize the One-norm. To build up this unitary, we start by the definition of the Orbital rotation operator which reads
$$ U(\vec{\kappa}) =\sum_{t>u}^{active} \kappa_{tu} (\hat{E}_{tu} - \hat{E}_{ut})$$
with $\hat{E}_{tu}=\frac{1} {\sqrt{2}}\sum_{\sigma}a^\dagger_{t,\sigma} a_{u,\sigma}$ and $\kappa_{tu}$ a rotation parameter affecting MOs $t$ and $u$ (note the absence of self rotation here with $t\neq u$). This operator takes the form of single excitations (with $t,u$ indices running only in the active space). Starting from this definition, a little bit of aglebra can show that implementing this rotation operator on creation/anihilation operators is strictly equivalent to redefining the MO coeff matrix of the Hamiltonian as follows
$$ \mathbf{C}_{new}(\vec{\kappa}) = \mathbf{C}_{ref} \mathbf{U}_{OO}(\vec{\kappa}) $$ 
Here, $\mathbf{U}_{OO}(\vec{\kappa})$ is a matrix representation of the effect of the rotations operator on the MOs. This matrix is an exponentiated version of the skew-matrix noted $\mathbf{K}$ encoding the parameter $\vec{\kappa}$. In other words
$$\mathbf{U}_{OO}(\vec{\kappa}) = e^{-\mathbf{K}} \text{,  with  } \mathbf{K} = skew(\vec{\kappa}) = \pmatrix{ 0 & \kappa_{01} & \kappa_{02} & \ldots & \kappa_{0N} \\
-\kappa_{01} & 0 & \kappa_{12} & \ldots & \kappa_{1N} \\
-\kappa_{02} & -\kappa_{12} & 0 &\ldots & \vdots \\
\vdots & \vdots & \vdots &\ddots & \vdots \\
-\kappa_{0N} & -\kappa_{1N} & \ldots & \ldots & 0 }$$

## Numerical implementation

Starting from this, we can encode this exponential numerically and use the kappa parameters (total number of parameters = $N(N-1)/2$) as degrees of freedom to reduce the one-norm. For this, first I use scimpy symbols to encode the matrix.

In [1]:
import psi4
import sys
import os
import openfermion
from openfermionpyscf import run_pyscf, compute_integrals
import numpy as np
import copy
from scipy.optimize import minimize
import math as m
import scipy 
import sympy
from sympy.matrices import zeros
from random import random
from openfermion import ops, InteractionOperator, get_sparse_operator, jordan_wigner
from openfermion.functionals import JW1norm_spat
from pyscf import gto, scf, lo, tools, fci, ao2mo, mcscf, df
import module as md
import time

# Set molecule parameters.
basis = ['sto-3g', 'cc-pvdz','def2svp'][0]
multiplicity = 1
# Set calculation parameters.
consider_cas = 1
n_orbitals = 8
n_electrons = 8
H_chain = 0
H_ring = 0
run_fci = 0
run_casci = 0
if H_chain:
    N_atoms = 14
    r = 1.8
    if run_fci:
        description = 'H'+str(N_atoms) + str(basis) + 'FCI'
    elif run_casci or consider_cas:
        description = 'H'+str(N_atoms) + str(basis) + 'ne' + str(n_electrons) +\
            'no' + str(n_orbitals)
    geometry = [('H',( 0., 0., z*r)) for z in range(N_atoms)]
    
elif H_ring:
    theta = 88 * np.pi/180
    r = 1.3
    # Introduction of the molecular structure (.txt file)
    geometry = [
                ('H', (r*np.cos(theta/2.),   r*np.sin(theta/2.),  0.)),
                ('H', (r*np.cos(theta/2.),   -r*np.sin(theta/2.), 0.)),
                ('H', (-r*np.cos(theta/2.),  r*np.sin(theta/2.),  0.)),
                ('H', (-r*np.cos(theta/2.),  -r*np.sin(theta/2.), 0.))
                ]
    if run_fci:
        description = 'H4' + str(basis) + 'FCI'
    elif run_casci or consider_cas:
        description = 'H4' + str(basis) + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = 'H4' + str(basis)

else:
    fname = ['xyz_files/H2nosym.txt','xyz_files/H2COnosym.txt','xyz_files/H10.txt',\
             'xyz_files/C2.txt', 'xyz_files/LiH.txt', 'xyz_files/HLiO.txt', \
             'xyz_files/H2Onosym.txt', 'xyz_files/H14.txt', \
             'xyz_files/hnch2_s0min_dzp.txt', 'xyz_files/hnc3h6_s0min_dzp.txt',\
             'xyz_files/hnc5h10_s0min_dzp.txt', 'xyz_files/hnc7h14_s0min_dzp.txt',\
             'xyz_files/benzene.txt','xyz_files/PCy3.txt','xyz_files/PCy3Cl2Ru.txt'][8]
    geometry = md.xyz_to_geom(fname)
    if run_fci:
        description = fname.replace('xyz_files/','').replace('.txt','') +\
            str(basis) + 'FCI'
    elif run_casci or consider_cas:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)\
            + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)

#description += localizemethod
nao = md.count_ao(geometry,basis,spin=multiplicity-1)
nelec = md.count_elec(geometry,basis,spin=multiplicity-1)
nmo = nao
print('nao:',nao,'\nnelec:', nelec)
threshold = 1e-10
if consider_cas:
    ncore = (nelec - n_electrons) // 2
    ntot = ncore + n_orbitals
    active_indices = list(range(ncore,ntot))
else:
    ncore = 0
    ntot = nmo
    active_indices = list(range(nmo))
mol = gto.Mole()
mol.atom = geometry
mol.spin = multiplicity - 1
mol.symmetry = 0
mol.build()

if multiplicity == 1:
    myhf = scf.RHF(mol)
else:
    myhf = scf.ROHF(mol)

myhf.run()

C_nonloc = np.copy(myhf.mo_coeff)

constant = float(mol.energy_nuc())


print('---------NON-LOCALIZED_ORBITALS---------')

C_copy = np.copy(C_nonloc)
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_copy, threshold)

t4 = time.time()
qub1norm = JW1norm_spat(np.copy(constant),
                        np.copy(one_body_integrals),
                        np.copy(two_body_integrals))
print('\ncalculating norm of qubit hamiltonian took', time.time()-t4)

print('\n')

print('---------LOCALIZED_ORBITALS_PM---------')
localizemethod = 'Pipek-Mezey'
localize = 1

C = np.copy(C_nonloc)
if localizemethod == 'Pipek-Mezey':
    orb = lo.PipekMezey(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'Boys':
    orb = lo.Boys(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'ibo':
    orb = lo.ibo.ibo(mol, C[:,ncore:ntot])
elif localizemethod == 'ER':
    orb = lo.EdmistonRuedenberg(mol).kernel(C[:,ncore:ntot])
else:
    raise ValueError('Localization method not recognized')

C_locPM = np.hstack((C_nonloc[:,:ncore],orb,C_nonloc[:,ntot:nmo]))
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_locPM, threshold)

t4 = time.time()
qub1norm_locPM = JW1norm_spat(np.copy(constant),
                              np.copy(one_body_integrals),
                              np.copy(two_body_integrals))
print('calculating norm of qubit hamiltonian took', time.time()-t4)

print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)




OPT_OO_MAX_ITER  = 50000 # Maximum number of steps for the OO

# Building the K matrix : generator for the MO rotations 
Rot_param_values = []
Rot_param_names  = []
bounds = (-1.,1.)

K_mat_sym = zeros( nmo )
bounds = []
for q in range(nmo-1):
    for p in range(q+1,nmo):
        if ( p in active_indices and q in active_indices ):
            K_mat_sym[ p, q ] = - sympy.Symbol( "K_{}{}".format( p, q ) )
            K_mat_sym[ q, p ] = - K_mat_sym[ p, q ] 
            Rot_param_names  += [sympy.Symbol( "K_{}{}".format( p, q ))]
            Rot_param_values += [0 * (random()-0.5)] # <== Can play on the efficiency of the OO, here 0.1 seems fine !
            bounds += [ [-1., 1.] ] 

# Building the bounds for the rotational parameter amplitudes in the form of constraints
# (simply beacause cobyla doesn't have intrinsic bound options)
cons = []
for factor in range(len(bounds)):
    lower, upper = bounds[factor]
    l = {'type': 'ineq',
         'fun': lambda x, lb=lower, i=factor: x[i] - lb}
    u = {'type': 'ineq',
         'fun': lambda x, ub=upper, i=factor: ub - x[i]}
    cons.append(l)
    cons.append(u)


nao: 13 
nelec: 16
converged SCF energy = -92.8222185440788
---------NON-LOCALIZED_ORBITALS---------

calculating norm of qubit hamiltonian took 0.02544093132019043


---------LOCALIZED_ORBITALS_PM---------
calculating norm of qubit hamiltonian took 0.02375936508178711
starting 1-norm nonloc is: 233.57225811991827
1norm locPM is: 210.26541298702293


In [2]:
K_mat_sym

Matrix([
[0, 0, 0, 0,      0,      0,      0,      0,      0,      0,       0,      0, 0],
[0, 0, 0, 0,      0,      0,      0,      0,      0,      0,       0,      0, 0],
[0, 0, 0, 0,      0,      0,      0,      0,      0,      0,       0,      0, 0],
[0, 0, 0, 0,      0,      0,      0,      0,      0,      0,       0,      0, 0],
[0, 0, 0, 0,      0,   K_54,   K_64,   K_74,   K_84,   K_94,   K_104,  K_114, 0],
[0, 0, 0, 0,  -K_54,      0,   K_65,   K_75,   K_85,   K_95,   K_105,  K_115, 0],
[0, 0, 0, 0,  -K_64,  -K_65,      0,   K_76,   K_86,   K_96,   K_106,  K_116, 0],
[0, 0, 0, 0,  -K_74,  -K_75,  -K_76,      0,   K_87,   K_97,   K_107,  K_117, 0],
[0, 0, 0, 0,  -K_84,  -K_85,  -K_86,  -K_87,      0,   K_98,   K_108,  K_118, 0],
[0, 0, 0, 0,  -K_94,  -K_95,  -K_96,  -K_97,  -K_98,      0,   K_109,  K_119, 0],
[0, 0, 0, 0, -K_104, -K_105, -K_106, -K_107, -K_108, -K_109,       0, K_1110, 0],
[0, 0, 0, 0, -K_114, -K_115, -K_116, -K_117, -K_118, -K_119, -K_1110,      0, 0],
[0, 0, 

In [4]:
K      = np.array( K_mat_sym.subs(list(zip(Rot_param_names,Rot_param_values))) ).astype(np.float64)  
U_OO   = scipy.linalg.expm( - K )

print("check identity:",np.allclose(np.eye(U_OO.shape[0]), U_OO @ U_OO.T))

check identity: True


Then I define a cost function to encode the evaluation of the One-Norm using the new MO coeff matrix. In the latter, we build the Hamiltonian with the new MO coeff. To proceed, we use $\mathbf{C}_{new}(\vec{\kappa})$ to build up the new electronic integrals.

In [5]:
def K_matr(Rot_param_values, active_indices):
    K = zeros( nmo )
    n = 0
    for q in range(nmo-1):
        for p in range(q+1,nmo):
            if ( p in active_indices and q in active_indices ):
                K[ p, q ] = - Rot_param_values[n]
                K[ q, p ] = - K[ p, q ]
                n += 1
    return np.array(K)

def Cost_function_OO_OneNorm(Rot_param_values):
#                              Rot_param_names,
#                              K_matr):
    """
    Cost function to minimize the One-Norm using MO rotations.
    """
    t6 = time.time()
    t5 = time.time()
    K = K_matr(Rot_param_values, active_indices)
    print("\nSubstituting variables took:",time.time()-t5)
    t1 = time.time()
    U_OO   = scipy.linalg.expm( - K )
    print("exponentiating matrix took:",time.time()-t1)
    t3 = time.time()
    if localize:
        C_MO   = C_locPM @ U_OO
    else:
        C_MO   = C_nonloc @ U_OO
    print("Matrix multiplication took:",time.time()-t3)
    t2 = time.time()
    one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_MO, threshold)
    print("extracting integrals took:",time.time()-t2)
    t4 = time.time()
    OneNorm = JW1norm_spat(np.copy(constant),
                           np.copy(one_body_integrals),
                           np.copy(two_body_integrals))
    print('calculating norm of qubit hamiltonian took', time.time()-t4)
    print('1-norm is:',OneNorm)
    print("total time for 1 cost function evaluation is", time.time()-t6)
    return OneNorm

print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)
t7 = time.time()
f_min_OO = minimize( Cost_function_OO_OneNorm,
                      x0      = Rot_param_values,
#                       args    = (Rot_param_names,
#                                  K_mat_sym),
                      constraints=cons,
                      method  = 'cobyla',
                      options = {'maxiter': OPT_OO_MAX_ITER,
                                'tol'    : 1e-5,
                                'disp': True}  )
print("total time for minimization with",OPT_OO_MAX_ITER,"max iterations was:", time.time()-t7)

[4, 5, 6, 7, 8, 9, 10, 11]
starting 1-norm nonloc is: 233.57225811991827
1norm locPM is: 210.26541298702293

Substituting variables took: 0.0019016265869140625
exponentiating matrix took: 0.0004489421844482422
Matrix multiplication took: 1.0967254638671875e-05
extracting integrals took: 0.03359818458557129
calculating norm of qubit hamiltonian took 0.03137969970703125
1-norm is: 210.26541298702293
total time for 1 cost function evaluation is 0.06770110130310059

Substituting variables took: 0.0024225711822509766
exponentiating matrix took: 0.0009350776672363281
Matrix multiplication took: 8.821487426757812e-06
extracting integrals took: 0.04232931137084961
calculating norm of qubit hamiltonian took 0.027941465377807617
1-norm is: 214.75692692121314
total time for 1 cost function evaluation is 0.07701349258422852

Substituting variables took: 0.002565145492553711
exponentiating matrix took: 0.000606536865234375
Matrix multiplication took: 1.2636184692382812e-05
extracting integrals took

extracting integrals took: 0.01897120475769043
calculating norm of qubit hamiltonian took 0.026142597198486328
1-norm is: 223.32264358106983
total time for 1 cost function evaluation is 0.04889059066772461

Substituting variables took: 0.0027909278869628906
exponentiating matrix took: 0.0006098747253417969
Matrix multiplication took: 1.1444091796875e-05
extracting integrals took: 0.033153533935546875
calculating norm of qubit hamiltonian took 0.02483081817626953
1-norm is: 223.4086365561288
total time for 1 cost function evaluation is 0.062139034271240234

Substituting variables took: 0.002575397491455078
exponentiating matrix took: 0.0006699562072753906
Matrix multiplication took: 1.0728836059570312e-05
extracting integrals took: 0.028734683990478516
calculating norm of qubit hamiltonian took 0.02733612060546875
1-norm is: 209.75025363475848
total time for 1 cost function evaluation is 0.06005263328552246

Substituting variables took: 0.002505779266357422
exponentiating matrix took: 0

extracting integrals took: 0.02178478240966797
calculating norm of qubit hamiltonian took 0.026742935180664062
1-norm is: 220.92118061431378
total time for 1 cost function evaluation is 0.05266547203063965

Substituting variables took: 0.0028073787689208984
exponentiating matrix took: 0.0007450580596923828
Matrix multiplication took: 9.298324584960938e-06
extracting integrals took: 0.024550437927246094
calculating norm of qubit hamiltonian took 0.024872541427612305
1-norm is: 209.27012146622337
total time for 1 cost function evaluation is 0.05383801460266113

Substituting variables took: 0.002856731414794922
exponentiating matrix took: 0.0010828971862792969
Matrix multiplication took: 8.58306884765625e-06
extracting integrals took: 0.022655248641967773
calculating norm of qubit hamiltonian took 0.02489471435546875
1-norm is: 219.07611577261503
total time for 1 cost function evaluation is 0.05187535285949707

Substituting variables took: 0.0038673877716064453
exponentiating matrix took:

extracting integrals took: 0.026981592178344727
calculating norm of qubit hamiltonian took 0.025914430618286133
1-norm is: 208.65457081995487
total time for 1 cost function evaluation is 0.05717587471008301

Substituting variables took: 0.0028946399688720703
exponentiating matrix took: 0.0007586479187011719
Matrix multiplication took: 1.0251998901367188e-05
extracting integrals took: 0.01748514175415039
calculating norm of qubit hamiltonian took 0.025552034378051758
1-norm is: 210.75641139852257
total time for 1 cost function evaluation is 0.047379493713378906

Substituting variables took: 0.0028693675994873047
exponentiating matrix took: 0.0007328987121582031
Matrix multiplication took: 9.5367431640625e-06
extracting integrals took: 0.017113685607910156
calculating norm of qubit hamiltonian took 0.024400711059570312
1-norm is: 207.51245350491251
total time for 1 cost function evaluation is 0.04580116271972656

Substituting variables took: 0.003535032272338867
exponentiating matrix too

extracting integrals took: 0.03974008560180664
calculating norm of qubit hamiltonian took 0.028010129928588867
1-norm is: 206.2987728109355
total time for 1 cost function evaluation is 0.07227468490600586

Substituting variables took: 0.003397703170776367
exponentiating matrix took: 0.0011391639709472656
Matrix multiplication took: 1.0251998901367188e-05
extracting integrals took: 0.048712730407714844
calculating norm of qubit hamiltonian took 0.023852109909057617
1-norm is: 206.92497990682782
total time for 1 cost function evaluation is 0.07732725143432617

Substituting variables took: 0.002911806106567383
exponentiating matrix took: 0.0011017322540283203
Matrix multiplication took: 9.775161743164062e-06
extracting integrals took: 0.042185068130493164
calculating norm of qubit hamiltonian took 0.028192996978759766
1-norm is: 205.84381594453305
total time for 1 cost function evaluation is 0.07463335990905762

Substituting variables took: 0.002955198287963867
exponentiating matrix took:

...

extracting integrals took: 0.025762319564819336
calculating norm of qubit hamiltonian took 0.022153139114379883
1-norm is: 204.30611201462204
total time for 1 cost function evaluation is 0.05233359336853027

Substituting variables took: 0.0026140213012695312
exponentiating matrix took: 0.0007381439208984375
Matrix multiplication took: 9.5367431640625e-06
extracting integrals took: 0.02788710594177246
calculating norm of qubit hamiltonian took 0.024904966354370117
1-norm is: 204.3060939029799
total time for 1 cost function evaluation is 0.05660533905029297

Substituting variables took: 0.0029077529907226562
exponentiating matrix took: 0.0008587837219238281
Matrix multiplication took: 9.298324584960938e-06
extracting integrals took: 0.026599884033203125
calculating norm of qubit hamiltonian took 0.027729272842407227
1-norm is: 204.30610803620257
total time for 1 cost function evaluation is 0.05895042419433594

Substituting variables took: 0.004009723663330078
exponentiating matrix took: 

calculating norm of qubit hamiltonian took 0.02699422836303711
1-norm is: 204.30605793788334
total time for 1 cost function evaluation is 0.07036399841308594

Substituting variables took: 0.0029251575469970703
exponentiating matrix took: 0.0019025802612304688
Matrix multiplication took: 9.5367431640625e-06
extracting integrals took: 0.035880088806152344
calculating norm of qubit hamiltonian took 0.028771162033081055
1-norm is: 204.30598676919328
total time for 1 cost function evaluation is 0.06972002983093262

Substituting variables took: 0.002968311309814453
exponentiating matrix took: 0.0011434555053710938
Matrix multiplication took: 9.5367431640625e-06
extracting integrals took: 0.035033464431762695
calculating norm of qubit hamiltonian took 0.02889108657836914
1-norm is: 204.30608003645486
total time for 1 cost function evaluation is 0.06827759742736816

Substituting variables took: 0.003040790557861328
exponentiating matrix took: 0.0011377334594726562
Matrix multiplication took: 9

calculating norm of qubit hamiltonian took 0.025264263153076172
1-norm is: 204.30559680914064
total time for 1 cost function evaluation is 0.06294488906860352

Substituting variables took: 0.003736257553100586
exponentiating matrix took: 0.0008423328399658203
Matrix multiplication took: 9.775161743164062e-06
extracting integrals took: 0.028423786163330078
calculating norm of qubit hamiltonian took 0.025769472122192383
1-norm is: 204.3055975953533
total time for 1 cost function evaluation is 0.059296607971191406

Substituting variables took: 0.0030739307403564453
exponentiating matrix took: 0.0008332729339599609
Matrix multiplication took: 8.106231689453125e-06
extracting integrals took: 0.025704622268676758
calculating norm of qubit hamiltonian took 0.025812625885009766
1-norm is: 204.3055985624498
total time for 1 cost function evaluation is 0.05587267875671387

Substituting variables took: 0.0031325817108154297
exponentiating matrix took: 0.0018744468688964844
Matrix multiplication t

calculating norm of qubit hamiltonian took 0.02748394012451172
1-norm is: 204.3055095891285
total time for 1 cost function evaluation is 0.05902409553527832

Substituting variables took: 0.012115001678466797
exponentiating matrix took: 0.0009732246398925781
Matrix multiplication took: 1.0251998901367188e-05
extracting integrals took: 0.0263826847076416
calculating norm of qubit hamiltonian took 0.02779531478881836
1-norm is: 204.30538129438378
total time for 1 cost function evaluation is 0.0675039291381836

Substituting variables took: 0.0029549598693847656
exponentiating matrix took: 0.0008485317230224609
Matrix multiplication took: 1.0013580322265625e-05
extracting integrals took: 0.041062116622924805
calculating norm of qubit hamiltonian took 0.025667667388916016
1-norm is: 204.30539553992276
total time for 1 cost function evaluation is 0.07100582122802734

Substituting variables took: 0.0028715133666992188
exponentiating matrix took: 0.0008330345153808594
Matrix multiplication took

calculating norm of qubit hamiltonian took 0.030077457427978516
1-norm is: 204.30527904324188
total time for 1 cost function evaluation is 0.08087849617004395

Substituting variables took: 0.0029757022857666016
exponentiating matrix took: 0.0008473396301269531
Matrix multiplication took: 9.775161743164062e-06
extracting integrals took: 0.040326595306396484
calculating norm of qubit hamiltonian took 0.029107093811035156
1-norm is: 204.30530480854242
total time for 1 cost function evaluation is 0.07349681854248047

Substituting variables took: 0.003790140151977539
exponentiating matrix took: 0.0013167858123779297
Matrix multiplication took: 1.1682510375976562e-05
extracting integrals took: 0.04105710983276367
calculating norm of qubit hamiltonian took 0.024193525314331055
1-norm is: 204.30525247021217
total time for 1 cost function evaluation is 0.07059431076049805

Substituting variables took: 0.0029327869415283203
exponentiating matrix took: 0.0008556842803955078
Matrix multiplication 

In [6]:
print("message:",f_min_OO.message,"number of function evaluations:",f_min_OO.nfev)

message: Optimization terminated successfully. nit: 1386


In [7]:
Cost_function_OO_OneNorm(f_min_OO.x)
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
C_OO = C_locPM @ U_OO
t13 = time.time()
for i in range(nmo):    
    tools.cubegen.orbital(mol, os.getcwd() + '/CUBE_FILES/pyscfcube'\
                          + description + 'onenorm_orb' + localizemethod + str(localize)\
                          + str(consider_cas) + str(i) , C_OO[:,i])
print('Cube files of molecule', description,'created in', os.getcwd() + '/CUBE_FILES/')
print('extracting cube files took', time.time()-t13)


Substituting variables took: 0.0029532909393310547
exponentiating matrix took: 0.0008616447448730469
Matrix multiplication took: 9.5367431640625e-06
extracting integrals took: 0.03643083572387695
calculating norm of qubit hamiltonian took 0.039260149002075195
1-norm is: 204.30523886581346
total time for 1 cost function evaluation is 0.08016276359558105
Cube files of molecule hnch2_s0min_dzpsto-3gne8no8 created in /home/emielkoridon/Master_Project/Python/Hamiltonian_terms/CUBE_FILES/
extracting cube files took 4.553831100463867
